<a href="https://colab.research.google.com/github/Rashi-Dwivedi1812/CodeQuery/blob/main/code_compiler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

In [5]:
import os
import json
import subprocess
import shutil
import glob
from openai import OpenAI
from google.colab import userdata # <-- Import userdata to access secrets

# =========================================================
#  LOADING KEY FROM SECRETS
# =========================================================
API_KEY = userdata.get('OPENAI_API_KEY')

# --- OpenAI Client Setup ---
client = None
if not API_KEY:
    print("="*50)
    print("ERROR: 'OPENAI_API_KEY' not found in Colab Secrets.")
    print("Please add it (see Cell 2) and restart the runtime.")
    print("="*50)
else:
    try:
        client = OpenAI(api_key=API_KEY)
        print("OpenAI client initialized successfully.")
    except Exception as e:
        print(f"Error initializing OpenAI client: {e}")

# --- SYSTEM PROMPT (Tells AI what to do) ---
SYSTEM_PROMPT = """You are an expert multi-purpose assistant.
First, analyze the user's prompt.

If the prompt is asking for code, return a JSON object with this structure:
{"type": "code", "language": "python" | "cpp" | "java" | "other", "code": "...", "explanation": "..."}
- "language": Specify the language as 'python', 'cpp', 'java', or 'other'.
- "code": The complete, runnable code. For C/C++/Java, this *must* include a main() function and necessary imports.
- **If Java, the main public class MUST be named 'Main'.**
- "explanation": A step-by-step list explaining the code.

If the prompt is a general question, return this JSON structure:
{"type": "text", "answer": "..."}
- "answer": The direct answer to the question.

Always use the JSON format.
"""

# --- NEW: Function to clean up old files ---
def cleanup_previous_files():
    """Removes files from the previous run."""
    files_to_remove = [
        "temp_code.py",
        "temp_code.cpp",
        "Main.java",
        "temp_exec"
    ]

    # Remove specific files
    for f in files_to_remove:
        if os.path.exists(f):
            os.remove(f)

    # Remove Java .class files
    for f in glob.glob("*.class"):
        os.remove(f)

    # Remove Python __pycache__
    if os.path.exists("__pycache__"):
        shutil.rmtree("__pycache__")

# --- UPDATED: Code Compilation Check ---
def check_code(language, code_string):
    """
    Cleans old files, saves and compiles new code.
    Returns (status, filename)
    """
    # Clean up files from the *last* run first
    cleanup_previous_files()

    filename = "temp_file"
    compile_cmd = ""
    status_msg = ""

    try:
        if language == 'python':
            filename = "temp_code.py"
            compile_cmd = f"python3 -m py_compile {filename}"

        elif language == 'cpp':
            filename = "temp_code.cpp"
            compile_cmd = f"g++ {filename} -o temp_exec"

        elif language == 'java':
            # Assumes AI names the class 'Main' as per system prompt
            filename = "Main.java"
            compile_cmd = f"javac {filename}"

        else:
            return (f"Cannot check code for language: {language}", "N/A")

        # Write the new code to the file
        with open(filename, 'w') as f:
            f.write(code_string)

        # Run the compile command
        result = subprocess.run(compile_cmd, shell=True, capture_output=True, text=True)

        # Check the result
        if result.returncode == 0:
            status_msg = "✅ Compilation Succeeded"
        else:
            status_msg = f"❌ Compilation FAILED:\n{result.stderr}"

        return (status_msg, filename)

    except Exception as e:
        return (f"An error occurred during check: {str(e)}", "N/A")


# --- Main Interactive Loop ---
if client:
    print("\n--- Interactive AI Assistant (with Code Check & File Saving) ---")
    print("Type your prompt and press Enter.")
    print("Type 'exit' to quit.\n")

    while True:
        user_prompt = input("Your Prompt: ")

        if user_prompt.lower() == 'exit':
            print("Exiting assistant.")
            break

        if not user_prompt:
            continue

        try:
            print("...Asking OpenAI...")

            # --- Direct call to OpenAI API ---
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {"role": "system", "content": SYSTEM_PROMPT},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0,
                response_format={"type": "json_object"}
            )

            response_content = completion.choices[0].message.content

            # --- Parse the JSON response ---
            response_data = json.loads(response_content)
            response_type = response_data.get('type')

            if response_type == 'code':
                # --- It's code, print all parts + compile status ---
                code_val = response_data.get('code')
                expl_val = response_data.get('explanation')
                lang_val = response_data.get('language')

                print("\n--- GENERATED CODE ---")
                print(code_val)
                print("------------------------")

                print("\n--- EXPLANATION ---")
                print(expl_val)
                print("------------------------")

                # --- NEW: Call the code checker ---
                print("\n--- COMPILATION STATUS ---")
                status, saved_filename = check_code(lang_val, code_val)
                print(status)

                # --- NEW: Print save location ---
                if saved_filename != "N/A":
                    print(f"\nFile saved as: {saved_filename}")
                    print("Find it in the 'Files' tab (folder icon 🗂️) on the left to download.")
                print("------------------------\n")

            elif response_type == 'text':
                # It's a general answer, just print the answer
                answer_val = response_data.get('answer')
                print("\n--- ANSWER ---")
                print(answer_val)
                print("----------------\n")

            else:
                # Fallback for unexpected response
                print("--- UNKNOWN RESPONSE ---")
                print(response_content)
                print("----------------------\n")

        except Exception as e:
            print(f"\n--- AN ERROR OCCURRED ---")
            print(f"Error details: {e}")
            print("------------------------\n")
else:
    print("Cannot start assistant. Please fix API key error.")

OpenAI client initialized successfully.

--- Interactive AI Assistant (with Code Check & File Saving) ---
Type your prompt and press Enter.
Type 'exit' to quit.

Your Prompt: c++ code to reverse a number
...Asking OpenAI...

--- GENERATED CODE ---
#include <iostream>
using namespace std;

int main() {
    int num, reversedNum = 0, remainder;
    cout << "Enter a number: ";
    cin >> num;
    while(num != 0) {
        remainder = num % 10;
        reversedNum = reversedNum * 10 + remainder;
        num /= 10;
    }
    cout << "Reversed number: " << reversedNum;
    return 0;
}
------------------------

--- EXPLANATION ---
1. Declare variables num, reversedNum, and remainder.
2. Prompt the user to enter a number and store it in 'num'.
3. Use a while loop to extract the last digit of 'num' and add it to 'reversedNum'.
4. Update 'num' by removing the last digit.
5. Repeat steps 3-4 until 'num' becomes 0.
6. Display the reversed number.
------------------------

--- COMPILATION STATUS ---